#### Pre-Processing

In [1]:
# Parse a PDF as well as a email invoice
from src.utils import preprocess_aor
pdf_path = "database/aor/AOR-sample1.pdf"
# preprocess_aor(aor_dir="database/aor")

#### Search & Query

In [2]:
# Chat with the preprocessed AOR for real-time chat based on RAG 
from src.aor import load_aors
from src.search import search_aor_with_item, search_aor_with_no


aor_list = load_aors()

# Quick routing: Item-based & Number-baseds

# item_query = "Hardware"
# matching_aors = search_aor_with_item(query, aor_list)

no_query = "DH928"
matching_aors = search_aor_with_no(no_query, aor_list)


print("Item-based AOR searching:")
for aor in matching_aors[:1]:
    print(f"AOR No: {aor.no}")
    print(f"Description: {aor.description}")
    print(f"Items: {aor.items}")
    print(f"Budgets: {aor.budgets}")
    print(f"Expiry Date: {aor.expiry_date}")
    print("---")

/opt/homebrew/anaconda3/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Item-based AOR searching:
AOR No: DH928/6/4
Description: Approval of requirement specifications for the implementation of Crew Mobile Digital Services Lite (CMDS-Lite) for First Flotilla, specifically for a Proof of Concept (PoC) trial on two Frigates (FFSes).
Items: ['Hardware for 2 FFSes', 'Software and Licenses for 2 FFSes', 'IT man-effort and Professional Services']
Budgets: [36000.0, 10000.0, 78000.0]
Expiry Date: 2024-05-16
---


#### RAG-based chat 

In [3]:
# How many budget is left in the AOR xxx 
# What kind of item is covered in AOR xxx
from src.search import query_detail, query_rough
query = "What does this AOR cover?"

# Detailed Query with access to full AOR text
# query_detail(aor, query)

# Rough Query with access to concise AOR summary
# query_rough(aor, query)

#### RAG Agent

In [4]:
from src.search import query_memory, Memory

class RagAgent:
    def __init__(self):
        self.memory = Memory()
        self.messages = []

    def chat(self, user_message):
        self.messages.append({"role": "user", "content": user_message})
        
        # Determine if we need to query the memory
        if self._should_query_memory(user_message):
            memory_query = self._generate_memory_query(user_message)
            memory_response, self.memory = query_memory(memory_query, self.memory)
            
            # Use the memory response to formulate the agent's response
            agent_response = self._formulate_response(user_message, memory_response)
        else:
            # Generate a response without querying memory
            agent_response = self._generate_response(user_message)
        
        self.messages.append({"role": "assistant", "content": agent_response})
        return agent_response

    def _should_query_memory(self, user_message):
        # Implement logic to determine if memory should be queried
        # This could be based on keywords, intent recognition, etc.
        return "AOR" in user_message or "budget" in user_message.lower()

    def _generate_memory_query(self, user_message):
        # Implement logic to generate an appropriate memory query
        # This could involve extracting key information from the user message
        return f"Find information related to: {user_message}"

    def _formulate_response(self, user_message, memory_response):
        # Implement logic to formulate a response based on the memory query result
        return f"Based on the information I found: {memory_response}"

    def _generate_response(self, user_message):
        # Implement logic for generating a response without querying memory
        return f"I understand you're asking about: {user_message}. How can I assist you further?"

    def get_conversation_history(self):
        return self.messages

# Example usage
agent = RagAgent()
user_message = "Tell me about AOR DH928"
response = agent.chat(user_message)
print(f"User: {user_message}")
print(f"Agent: {response}")

user_message = "What's the weather like today?"
response = agent.chat(user_message)
print(f"\nUser: {user_message}")
print(f"Agent: {response}")

print("\nConversation History:")
for message in agent.get_conversation_history():
    print(f"{message['role'].capitalize()}: {message['content']}")
    
    